In [1]:
import numpy as np
from numpy import array, dot, diag, reshape, transpose
from scipy.linalg import eigvalsh
from scipy.integrate import odeint, ode
from math import pi
from math import factorial

from sys import argv

/Users/shaneblade/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def construct_basis_2B(holes, particles):
  basis = []
  for i in holes:
    for j in holes:
      basis.append((i, j))

  for i in holes:
    for a in particles:
      basis.append((i, a))

  for a in particles:
    for i in holes:
      basis.append((a, i))

  for a in particles:
    for b in particles:
      basis.append((a, b))

  return basis


def construct_basis_ph2B(holes, particles):
  basis = []
  for i in holes:
    for j in holes:
      basis.append((i, j))

  for i in holes:
    for a in particles:
      basis.append((i, a))

  for a in particles:
    for i in holes:
      basis.append((a, i))

  for a in particles:
    for b in particles:
      basis.append((a, b))

  return basis


#
# We use dictionaries for the reverse lookup of state indices
#
def construct_index_2B(bas2B):
  index = { }
  for i, state in enumerate(bas2B):
    index[state] = i

  return index

In [3]:
#-----------------------------------------------------------------------------------
# transform matrices to particle-hole representation
#-----------------------------------------------------------------------------------
def ph_transform_2B(Gamma, bas2B, idx2B, basph2B, idxph2B):
  dim = len(basph2B)
  Gamma_ph = np.zeros((dim, dim))

  for i1, (a,b) in enumerate(basph2B):
    for i2, (c, d) in enumerate(basph2B):
      Gamma_ph[i1, i2] -= Gamma[idx2B[(a,d)], idx2B[(c,b)]]

  return Gamma_ph

def inverse_ph_transform_2B(Gamma_ph, bas2B, idx2B, basph2B, idxph2B):
  dim = len(bas2B)
  Gamma = np.zeros((dim, dim))

  for i1, (a,b) in enumerate(bas2B):
    for i2, (c, d) in enumerate(bas2B):
      Gamma[i1, i2] -= Gamma_ph[idxph2B[(a,d)], idxph2B[(c,b)]]
  
  return Gamma

In [4]:
#-----------------------------------------------------------------------------------
# commutator of matrices
#-----------------------------------------------------------------------------------
def commutator(a,b):
  return dot(a,b) - dot(b,a)

#-----------------------------------------------------------------------------------
# norms of off-diagonal Hamiltonian pieces
#-----------------------------------------------------------------------------------
def calc_fod_norm(f, user_data):
  particles = user_data["particles"]
  holes     = user_data["holes"]
  
  norm = 0.0
  for a in particles:
    for i in holes:
      norm += f[a,i]**2 + f[i,a]**2

  return np.sqrt(norm)

def calc_Gammaod_norm(Gamma, user_data):
  particles = user_data["particles"]
  holes     = user_data["holes"]
  idx2B     = user_data["idx2B"]

  norm = 0.0
  for a in particles:    
    for b in particles:
      for i in holes:
        for j in holes:
          norm += Gamma[idx2B[(a,b)],idx2B[(i,j)]]**2 + Gamma[idx2B[(i,j)],idx2B[(a,b)]]**2

  return np.sqrt(norm)

In [5]:
#-----------------------------------------------------------------------------------
# occupation number matrices
#-----------------------------------------------------------------------------------
def construct_occupation_1B(bas1B, holes, particles):
  dim = len(bas1B)
  occ = np.zeros(dim)

  for i in holes:
    occ[i] = 1.

  return occ

# diagonal matrix: n_a - n_b
def construct_occupationA_2B(bas2B, occ1B):
  dim = len(bas2B)
  occ = np.zeros((dim,dim))

  for i1, (i,j) in enumerate(bas2B):
    occ[i1, i1] = occ1B[i] - occ1B[j]

  return occ


# diagonal matrix: 1 - n_a - n_b
def construct_occupationB_2B(bas2B, occ1B):
  dim = len(bas2B)
  occ = np.zeros((dim,dim))

  for i1, (i,j) in enumerate(bas2B):
    occ[i1, i1] = 1. - occ1B[i] - occ1B[j]

  return occ

# diagonal matrix: n_a * n_b
def construct_occupationC_2B(bas2B, occ1B):
  dim = len(bas2B)
  occ = np.zeros((dim,dim))

  for i1, (i,j) in enumerate(bas2B):
    occ[i1, i1] = occ1B[i] * occ1B[j]

  return occ

In [6]:
def eta_wegner(f, Gamma, user_data):

  dim1B     = user_data["dim1B"]
  holes     = user_data["holes"]
  particles = user_data["particles"]
  bas2B     = user_data["bas2B"]
  basph2B   = user_data["basph2B"]
  idx2B     = user_data["idx2B"]
  idxph2B   = user_data["idxph2B"]
  occB_2B   = user_data["occB_2B"]
  occC_2B   = user_data["occC_2B"]
  occphA_2B = user_data["occphA_2B"]


  # split Hamiltonian in diagonal and off-diagonal parts
  fd      = np.zeros_like(f)
  fod     = np.zeros_like(f)
  Gammad  = np.zeros_like(Gamma)
  Gammaod = np.zeros_like(Gamma)

  for a in particles:
    for i in holes:
      fod[a, i] = f[a,i]
      fod[i, a] = f[i,a]
  fd = f - fod

  for a in particles:
    for b in particles:
      for i in holes:
        for j in holes:
          Gammaod[idx2B[(a,b)], idx2B[(i,j)]] = Gamma[idx2B[(a,b)], idx2B[(i,j)]]
          Gammaod[idx2B[(i,j)], idx2B[(a,b)]] = Gamma[idx2B[(i,j)], idx2B[(a,b)]]
  Gammad = Gamma - Gammaod

 #############################        
  # one-body part of the generator
  eta1B  = np.zeros_like(f)

  # 1B - 1B
  eta1B += commutator(fd, fod)

  # 1B - 2B
  for p in range(dim1B):
    for q in range(dim1B):
      for i in holes:
        for a in particles:
          eta1B[p,q] += (
            fd[i,a]  * Gammaod[idx2B[(a, p)], idx2B[(i, q)]] 
            - fd[a,i]  * Gammaod[idx2B[(i, p)], idx2B[(a, q)]] 
            - fod[i,a] * Gammad[idx2B[(a, p)], idx2B[(i, q)]] 
            + fod[a,i] * Gammad[idx2B[(i, p)], idx2B[(a, q)]]
          )

  # 2B - 2B
  # n_a n_b nn_c + nn_a nn_b n_c = n_a n_b + (1 - n_a - n_b) * n_c
  GammaGamma = dot(Gammad, dot(occB_2B, Gammaod))
  for p in range(dim1B):
    for q in range(dim1B):
      for i in holes:
        eta1B[p,q] += 0.5*(
          GammaGamma[idx2B[(i,p)], idx2B[(i,q)]] 
          - transpose(GammaGamma)[idx2B[(i,p)], idx2B[(i,q)]]
        )

  GammaGamma = dot(Gammad, dot(occC_2B, Gammaod))
  for p in range(dim1B):
    for q in range(dim1B):
      for r in range(dim1B):
        eta1B[p,q] += 0.5*(
          GammaGamma[idx2B[(r,p)], idx2B[(r,q)]] 
          - transpose(GammaGamma)[idx2B[(r,p)], idx2B[(r,q)]] 
        )


  #############################        
  # two-body flow equation  
  eta2B = np.zeros_like(Gamma)

  # 1B - 2B
  for p in range(dim1B):
    for q in range(dim1B):
      for r in range(dim1B):
        for s in range(dim1B):
          for t in range(dim1B):
            eta2B[idx2B[(p,q)],idx2B[(r,s)]] += (
              fd[p,t] * Gammaod[idx2B[(t,q)],idx2B[(r,s)]] 
              + fd[q,t] * Gammaod[idx2B[(p,t)],idx2B[(r,s)]] 
              - fd[t,r] * Gammaod[idx2B[(p,q)],idx2B[(t,s)]] 
              - fd[t,s] * Gammaod[idx2B[(p,q)],idx2B[(r,t)]]
              - fod[p,t] * Gammad[idx2B[(t,q)],idx2B[(r,s)]] 
              - fod[q,t] * Gammad[idx2B[(p,t)],idx2B[(r,s)]] 
              + fod[t,r] * Gammad[idx2B[(p,q)],idx2B[(t,s)]] 
              + fod[t,s] * Gammad[idx2B[(p,q)],idx2B[(r,t)]]
            )

  
  # 2B - 2B - particle and hole ladders
  # Gammad.occB.Gammaod
  GammaGamma = dot(Gammad, dot(occB_2B, Gammaod))

  eta2B += 0.5 * (GammaGamma - transpose(GammaGamma))

  # 2B - 2B - particle-hole chain
  
  # transform matrices to particle-hole representation and calculate 
  # Gammad_ph.occA_ph.Gammaod_ph
  Gammad_ph = ph_transform_2B(Gammad, bas2B, idx2B, basph2B, idxph2B)
  Gammaod_ph = ph_transform_2B(Gammaod, bas2B, idx2B, basph2B, idxph2B)

  GammaGamma_ph = dot(Gammad_ph, dot(occphA_2B, Gammaod_ph))

  # transform back to standard representation
  GammaGamma    = inverse_ph_transform_2B(GammaGamma_ph, bas2B, idx2B, basph2B, idxph2B)

  # commutator / antisymmetrization
  work = np.zeros_like(GammaGamma)
  for i1, (i,j) in enumerate(bas2B):
    for i2, (k,l) in enumerate(bas2B):
      work[i1, i2] -= (
        GammaGamma[i1, i2] 
        - GammaGamma[idx2B[(j,i)], i2] 
        - GammaGamma[i1, idx2B[(l,k)]] 
        + GammaGamma[idx2B[(j,i)], idx2B[(l,k)]]
      )
  GammaGamma = work

  eta2B += GammaGamma


  return eta1B, eta2B

In [7]:
#-----------------------------------------------------------------------------------
# derivatives 
#-----------------------------------------------------------------------------------
#def flow_imsrg2(eta1B, eta2B, f, Gamma, user_data):
    
def symmetric_comm(A, B):
    
  dim1B     = user_data["dim1B"]
  holes     = user_data["holes"]
  particles = user_data["particles"]
  bas2B     = user_data["bas2B"]
  idx2B     = user_data["idx2B"]
  basph2B   = user_data["basph2B"]
  idxph2B   = user_data["idxph2B"]
  occB_2B   = user_data["occB_2B"]
  occC_2B   = user_data["occC_2B"]
  occphA_2B = user_data["occphA_2B"]

  #############################        
  # zero-body flow equation

  # print(np.linalg.norm(eta2B+eta2B.T))  
  A0B, A1B, A2B = get_operator_from_y(A, dim1B, dim2B)
  B0B, B1B, B2B = get_operator_from_y(B, dim1B, dim2B)
  
  C0B = 0.0

  for i in holes:
    for a in particles:
      C0B += A1B[i,a] * B1B[a,i] - B1B[a,i] * A1B[i,a]

  for i in holes:
    for j in holes:
      for a in particles:
        for b in particles:
          C0B += 0.5 * A2B[idx2B[(i,j)], idx2B[(a,b)]] * B2B[idx2B[(a,b)], idx2B[(i,j)]]
        
  #print("C0B =", C0B)


  #############################        
  # one-body flow equation  
  C1B  = np.zeros_like(A1B)

  # 1B - 1B
  C1B += commutator(B1B, A1B)

  # 1B - 2B
  for p in range(dim1B):
    for q in range(dim1B):
      for i in holes:
        for a in particles:
          C1B[p,q] += (
            A1B[i,a] * B2B[idx2B[(a, p)], idx2B[(i, q)]] 
            - A1B[a,i] * B2B[idx2B[(i, p)], idx2B[(a, q)]] 
            - B1B[i,a] * A2B[idx2B[(a, p)], idx2B[(i, q)]] 
            + B1B[a,i] * A2B[idx2B[(i, p)], idx2B[(a, q)]]
          )

  # 2B - 2B
  # n_a n_b nn_c + nn_a nn_b n_c = n_a n_b + (1 - n_a - n_b) * n_c
  AB = dot(A2B, dot(occB_2B, B2B))
  for p in range(dim1B):
    for q in range(dim1B):
      for i in holes:
        C1B[p,q] += 0.5*(
          AB[idx2B[(i,p)], idx2B[(i,q)]] 
          + transpose(AB)[idx2B[(i,p)], idx2B[(i,q)]]
        )

  AB = dot(A2B, dot(occC_2B, B2B))
  for p in range(dim1B):
    for q in range(dim1B):
      for r in range(dim1B):
        C1B[p,q] += 0.5*(
          AB[idx2B[(r,p)], idx2B[(r,q)]] 
          + transpose(AB)[idx2B[(r,p)], idx2B[(r,q)]] 
        )
  #print("C1B = ", C1B[:dim1B,:dim1B])
  #print("C1B symmetry:", np.linalg.norm(C1B-transpose(C1B)))

  #############################        
  # two-body flow equation  
  C2B = np.zeros_like(A2B)

  # 1B - 2B
  for p in range(dim1B):
    for q in range(dim1B):
      for r in range(dim1B):
        for s in range(dim1B):
          for t in range(dim1B):
            C2B[idx2B[(p,q)],idx2B[(r,s)]] += (
              A1B[p,t] * B2B[idx2B[(t,q)],idx2B[(r,s)]] 
              + A1B[q,t] * B2B[idx2B[(p,t)],idx2B[(r,s)]] 
              - A1B[t,r] * B2B[idx2B[(p,q)],idx2B[(t,s)]] 
              - A1B[t,s] * B2B[idx2B[(p,q)],idx2B[(r,t)]]
              - B1B[p,t] * A2B[idx2B[(t,q)],idx2B[(r,s)]] 
              - B1B[q,t] * A2B[idx2B[(p,t)],idx2B[(r,s)]] 
              + B1B[t,r] * A2B[idx2B[(p,q)],idx2B[(t,s)]] 
              + B1B[t,s] * A2B[idx2B[(p,q)],idx2B[(r,t)]]
            )

  
  # 2B - 2B - particle and hole ladders
  # eta2B.occB.Gamma
  AB = dot(A2B, dot(occB_2B, B2B))

  C2B += 0.5 * (AB + transpose(AB))

  # 2B - 2B - particle-hole chain
  
  # transform matrices to particle-hole representation and calculate 
  # eta2B_ph.occA_ph.Gamma_ph
  A2B_ph = ph_transform_2B(A2B, bas2B, idx2B, basph2B, idxph2B)
  B2B_ph = ph_transform_2B(B2B, bas2B, idx2B, basph2B, idxph2B)

  AB_ph = dot(A2B_ph, dot(occphA_2B, B2B_ph))

  # transform back to standard representation
  AB    = inverse_ph_transform_2B(AB_ph, bas2B, idx2B, basph2B, idxph2B)

  # commutator / antisymmetrization
  work = np.zeros_like(AB)
  for i1, (i,j) in enumerate(bas2B):
    for i2, (k,l) in enumerate(bas2B):
      work[i1, i2] -= (
        AB[i1, i2] 
        - AB[idx2B[(j,i)], i2] 
        - AB[i1, idx2B[(l,k)]] 
        + AB[idx2B[(j,i)], idx2B[(l,k)]]
      )
  AB = work

  C2B += AB
 
  #print("C2B = ", C2B[:10,:10])
  #print("C2B symmetry:", np.linalg.norm(C2B-transpose(C2B)))
  
  y_C = np.append([C0B], np.append(reshape(C1B, -1), reshape(C2B, -1)))
#   print("y_C=", hex(id(y_C)))

  return y_C

In [8]:
#-----------------------------------------------------------------------------------
# derivatives 
#-----------------------------------------------------------------------------------
#def flow_imsrg2(eta1B, eta2B, f, Gamma, user_data):
    
def antisymm_comm(A, B):
    
  dim1B     = user_data["dim1B"]
  holes     = user_data["holes"]
  particles = user_data["particles"]
  bas2B     = user_data["bas2B"]
  idx2B     = user_data["idx2B"]
  basph2B   = user_data["basph2B"]
  idxph2B   = user_data["idxph2B"]
  occB_2B   = user_data["occB_2B"]
  occC_2B   = user_data["occC_2B"]
  occphA_2B = user_data["occphA_2B"]

  #############################        
  # zero-body flow equation

  # print(np.linalg.norm(eta2B+eta2B.T))  
  A0B, A1B, A2B = get_operator_from_y(A, dim1B, dim2B)
  B0B, B1B, B2B = get_operator_from_y(B, dim1B, dim2B)
  
  C0B = 0.0

  for i in holes:
    for a in particles:
      C0B += A1B[i, a] * B1B[a,i] - B1B[i,a] * A1B[a,i]

  for i in holes:
    for j in holes:
      for a in particles:
        for b in particles:
          C0B += 0.25 * A2B[idx2B[(i,j)], idx2B[(a,b)]] * B2B[idx2B[(a,b)], idx2B[(i,j)]]-0.25 * B2B[idx2B[(i,j)], idx2B[(a,b)]] * A2B[idx2B[(a,b)], idx2B[(i,j)]]   

#   print("C0B(as) =", C0B)

  #############################        
  # one-body flow equation  
  C1B  = np.zeros_like(A1B)

  # 1B - 1B
  C1B += commutator(A1B, B1B)

  # 1B - 2B
  for p in range(dim1B):
    for q in range(dim1B):
      for i in holes:
        for a in particles:
          C1B[p,q] += (
            A1B[i,a] * B2B[idx2B[(a, p)], idx2B[(i, q)]] 
            - A1B[a,i] * B2B[idx2B[(i, p)], idx2B[(a, q)]] 
            - B1B[i,a] * A2B[idx2B[(a, p)], idx2B[(i, q)]] 
            + B1B[a,i] * A2B[idx2B[(i, p)], idx2B[(a, q)]]
          )

  # 2B - 2B
  # n_a n_b nn_c + nn_a nn_b n_c = n_a n_b + (1 - n_a - n_b) * n_c
  AB = dot(A2B, dot(occB_2B, B2B))
  for p in range(dim1B):
    for q in range(dim1B):
      for i in holes:
        C1B[p,q] += 0.5*(
          AB[idx2B[(i,p)], idx2B[(i,q)]] 
          - transpose(AB)[idx2B[(i,p)], idx2B[(i,q)]]
        )

  AB = dot(A2B, dot(occC_2B, B2B))
  for p in range(dim1B):
    for q in range(dim1B):
      for r in range(dim1B):
        C1B[p,q] += 0.5*(
          AB[idx2B[(r,p)], idx2B[(r,q)]] 
          - transpose(AB)[idx2B[(r,p)], idx2B[(r,q)]] 
        )

  #print("C1B(as) =", C1B[:10,:10])
#   print("C1B(as) antisymmetry:", np.linalg.norm(C1B+transpose(C1B)))

  #############################        
  # two-body flow equation  
  C2B = np.zeros_like(A2B)

  # 1B - 2B
  for p in range(dim1B):
    for q in range(dim1B):
      for r in range(dim1B):
        for s in range(dim1B):
          for t in range(dim1B):
            C2B[idx2B[(p,q)],idx2B[(r,s)]] += (
              A1B[p,t] * B2B[idx2B[(t,q)],idx2B[(r,s)]] 
              + A1B[q,t] * B2B[idx2B[(p,t)],idx2B[(r,s)]] 
              - A1B[t,r] * B2B[idx2B[(p,q)],idx2B[(t,s)]] 
              - A1B[t,s] * B2B[idx2B[(p,q)],idx2B[(r,t)]]
              - B1B[p,t] * A2B[idx2B[(t,q)],idx2B[(r,s)]] 
              - B1B[q,t] * A2B[idx2B[(p,t)],idx2B[(r,s)]] 
              + B1B[t,r] * A2B[idx2B[(p,q)],idx2B[(t,s)]] 
              + B1B[t,s] * A2B[idx2B[(p,q)],idx2B[(r,t)]]
            )

  
  # 2B - 2B - particle and hole ladders
  # eta2B.occB.Gamma
  AB = dot(A2B, dot(occB_2B, B2B))

  C2B += 0.5 * (AB - transpose(AB))

  # 2B - 2B - particle-hole chain
  
  # transform matrices to particle-hole representation and calculate 
  # eta2B_ph.occA_ph.Gamma_ph
  A2B_ph = ph_transform_2B(A2B, bas2B, idx2B, basph2B, idxph2B)
  B2B_ph = ph_transform_2B(B2B, bas2B, idx2B, basph2B, idxph2B)

  AB_ph = dot(A2B_ph, dot(occphA_2B, B2B_ph))

  # transform back to standard representation
  AB    = inverse_ph_transform_2B(AB_ph, bas2B, idx2B, basph2B, idxph2B)

  # commutator / antisymmetrization
  work = np.zeros_like(AB)
  for i1, (i,j) in enumerate(bas2B):
    for i2, (k,l) in enumerate(bas2B):
      work[i1, i2] -= (
        AB[i1, i2] 
        - AB[idx2B[(j,i)], i2] 
        - AB[i1, idx2B[(l,k)]] 
        + AB[idx2B[(j,i)], idx2B[(l,k)]]
      )
  AB = work

  C2B += AB
  #print("C2B(as) = ", C2B[:10,:10])
#   print("C2B(as) antisymmetry:", np.linalg.norm(C2B+transpose(C2B)))

  y_C = np.append([C0B], np.append(reshape(C1B, -1), reshape(C2B, -1)))
#   print("y_C(as)=", hex(id(y_C)))

  return y_C

In [9]:
#-----------------------------------------------------------------------------------
# derivative wrapper
#-----------------------------------------------------------------------------------
def get_operator_from_y(y, dim1B, dim2B):
  
  # reshape the solution vector into 0B, 1B, 2B pieces
  ptr = 0
  zero_body = y[ptr]

  ptr += 1
  one_body = reshape(y[ptr:ptr+dim1B*dim1B], (dim1B, dim1B))

  ptr += dim1B*dim1B
  two_body = reshape(y[ptr:ptr+dim2B*dim2B], (dim2B, dim2B))

  return zero_body,one_body,two_body


def derivative_wrapper(t, y, user_data):

  dim1B = user_data["dim1B"]
  dim2B = dim1B*dim1B


  holes     = user_data["holes"]
  particles = user_data["particles"]
  bas1B     = user_data["bas1B"]
  bas2B     = user_data["bas2B"]
  basph2B   = user_data["basph2B"]
  idx2B     = user_data["idx2B"]
  idxph2B   = user_data["idxph2B"]
  occA_2B   = user_data["occA_2B"]
  occB_2B   = user_data["occB_2B"]
  occC_2B   = user_data["occC_2B"]
  occphA_2B = user_data["occphA_2B"]
  calc_eta  = user_data["calc_eta"]
  calc_rhs  = user_data["calc_rhs"]

  # extract operator pieces from solution vector
  E, f, Gamma = get_operator_from_y(y, dim1B, dim2B)


  # calculate the generator
  eta1B, eta2B = calc_eta(f, Gamma, user_data)

  # calculate the right-hand side
  dE, df, dGamma = calc_rhs(eta1B, eta2B, f, Gamma, user_data)

  # convert derivatives into linear array
  dy   = np.append([dE], np.append(reshape(df, -1), reshape(dGamma, -1)))

  # share data
  user_data["dE"] = dE
  user_data["eta_norm"] = np.linalg.norm(eta1B,ord='fro')+np.linalg.norm(eta2B,ord='fro')
  
  return dy

#-----------------------------------------------------------------------------------
# pairing Hamiltonian
#-----------------------------------------------------------------------------------
def pairing_hamiltonian(delta, g, user_data):
  bas1B = user_data["bas1B"]
  bas2B = user_data["bas2B"]
  idx2B = user_data["idx2B"]

  dim = len(bas1B)
  H1B = np.zeros((dim,dim))

  for i in bas1B:
    H1B[i,i] = delta*np.floor_divide(i, 2)

  dim = len(bas2B)
  H2B = np.zeros((dim, dim))

  # spin up states have even indices, spin down the next odd index
  for (i, j) in bas2B:
    if (i % 2 == 0 and j == i+1):
      for (k, l) in bas2B:
        if (k % 2 == 0 and l == k+1):
          H2B[idx2B[(i,j)],idx2B[(k,l)]] = -0.5*g
          H2B[idx2B[(j,i)],idx2B[(k,l)]] = 0.5*g
          H2B[idx2B[(i,j)],idx2B[(l,k)]] = 0.5*g
          H2B[idx2B[(j,i)],idx2B[(l,k)]] = -0.5*g
  
  return H1B, H2B

In [10]:
#-----------------------------------------------------------------------------------
# normal-ordered pairing Hamiltonian
#-----------------------------------------------------------------------------------
def normal_order(H1B, H2B, user_data):
  bas1B     = user_data["bas1B"]
  bas2B     = user_data["bas2B"]
  idx2B     = user_data["idx2B"]
  particles = user_data["particles"]
  holes     = user_data["holes"]

  # 0B part
  E = 0.0
  for i in holes:
    E += H1B[i,i]

  for i in holes:
    for j in holes:
      E += 0.5*H2B[idx2B[(i,j)],idx2B[(i,j)]]  

  # 1B part
  f = H1B.copy()
  for i in bas1B:
    for j in bas1B:
      for h in holes:
        f[i,j] += H2B[idx2B[(i,h)],idx2B[(j,h)]]  

  # 2B part
  Gamma = H2B.copy()

  return E, f, Gamma


In [11]:
#-----------------------------------------------------------------------------------
# Perturbation theory
#-----------------------------------------------------------------------------------
def calc_mbpt2(f, Gamma, user_data):
  DE2 = 0.0

  particles = user_data["particles"]
  holes     = user_data["holes"]
  idx2B     = user_data["idx2B"]

  for i in holes:
    for j in holes:
      for a in particles:
        for b in particles:
          denom = f[i,i] + f[j,j] - f[a,a] - f[b,b]
          me    = Gamma[idx2B[(a,b)],idx2B[(i,j)]]
          DE2  += 0.25*me*me/denom

  return DE2

def calc_mbpt3(f, Gamma, user_data):
  particles = user_data["particles"]
  holes     = user_data["holes"]
  idx2B     = user_data["idx2B"]

  # DE3 = 0.0

  DE3pp = 0.0
  DE3hh = 0.0
  DE3ph = 0.0

  for a in particles:
    for b in particles:
      for c in particles:
        for d in particles:
          for i in holes:
            for j in holes:
              denom = (f[i,i] + f[j,j] - f[a,a] - f[b,b])*(f[i,i] + f[j,j] - f[c,c] - f[d,d])
              me    = Gamma[idx2B[(i,j)],idx2B[(a,b)]]*Gamma[idx2B[(a,b)],idx2B[(c,d)]]*Gamma[idx2B[(c,d)],idx2B[(i,j)]]
              DE3pp += 0.125*me/denom

  for i in holes:
    for j in holes:
      for k in holes:
        for l in holes:
          for a in particles:
            for b in particles:
              denom = (f[i,i] + f[j,j] - f[a,a] - f[b,b])*(f[k,k] + f[l,l] - f[a,a] - f[b,b])
              me    = Gamma[idx2B[(a,b)],idx2B[(k,l)]]*Gamma[idx2B[(k,l)],idx2B[(i,j)]]*Gamma[idx2B[(i,j)],idx2B[(a,b)]]
              DE3hh += 0.125*me/denom

  for i in holes:
    for j in holes:
      for k in holes:
        for a in particles:
          for b in particles:
            for c in particles:
              denom = (f[i,i] + f[j,j] - f[a,a] - f[b,b])*(f[k,k] + f[j,j] - f[a,a] - f[c,c])
              me    = Gamma[idx2B[(i,j)],idx2B[(a,b)]]*Gamma[idx2B[(k,b)],idx2B[(i,c)]]*Gamma[idx2B[(a,c)],idx2B[(k,j)]]
              DE3ph -= me/denom
  return DE3pp+DE3hh+DE3ph

In [12]:
#-----------------------------------------------------------------------------------
# magnus expansion code
#-----------------------------------------------------------------------------------
# grab delta and g from the command line

delta      = float(1.0)
g          = float(0.5)

particles  = 4

# setup shared data
dim1B     = 8

# this defines the reference state
# 1st state
holes     = [0,1,2,3]
particles = [4,5,6,7]

# 2nd state
# holes     = [0,1,4,5]
# particles = [2,3,6,7]

# 3rd state
# holes     = [0,1,6,7]
# particles = [2,3,4,5]

# basis definitions
bas1B     = range(dim1B)
bas2B     = construct_basis_2B(holes, particles)
basph2B   = construct_basis_ph2B(holes, particles)

idx2B     = construct_index_2B(bas2B)
idxph2B   = construct_index_2B(basph2B)

# occupation number matrices
occ1B     = construct_occupation_1B(bas1B, holes, particles)
occA_2B   = construct_occupationA_2B(bas2B, occ1B)
occB_2B   = construct_occupationB_2B(bas2B, occ1B)
occC_2B   = construct_occupationC_2B(bas2B, occ1B)

occphA_2B = construct_occupationA_2B(basph2B, occ1B)

# store shared data in a dictionary, so we can avoid passing the basis
# lookups etc. as separate parameters all the time
user_data  = {
"dim1B":      dim1B, 
"holes":      holes,
"particles":  particles,
"bas1B":      bas1B,
"bas2B":      bas2B,
"basph2B":    basph2B,
"idx2B":      idx2B,
"idxph2B":    idxph2B,
"occ1B":      occ1B,
"occA_2B":    occA_2B,
"occB_2B":    occB_2B,
"occC_2B":    occC_2B,
"occphA_2B":  occphA_2B,

"eta_norm":   0.0,                # variables for sharing data between ODE solver
"dE":         0.0#,                # and main routine


#"calc_eta":   eta_wegner,          # specify the generator (function object)
#"calc_rhs":   flow_imsrg2         # specify the right-hand side and truncation
}

dim1B = user_data["dim1B"]
dim2B = dim1B * dim1B

omega_0B = 0
omega_1B = np.zeros((dim1B, dim1B))  
omega_2B = np.zeros((dim2B, dim2B))

In [13]:
  bernoulli_arr = [1.,        -0.5,         0.16666667,  0.,         -0.03333333,  0.,
                   
    0.02380952,  0.,         -0.03333333,  0.   ]
    
  H1B, H2B = pairing_hamiltonian(delta, g, user_data)

In [14]:
  def recursive_func(A, B, deriv):
      if deriv == 0:
          return B
      else:
          result = symmetric_comm(A, recursive_func(A, B, deriv - 1))
          return result 

In [15]:
  def recursive_func_alt(A, B, deriv):
      if deriv == 0:
          return B
      else:
          result = antisymm_comm(A, recursive_func_alt(A, B, deriv - 1))
          return result 

In [22]:
  def magnus_stepper(y_omega, E0, f0, Gamma0, deriv_value):
    
      omega_0B, omega_1B, omega_2B = get_operator_from_y(y_omega, dim1B, dim2B) 
      #print("O0B=", omega_0B)
      #print("O1B=", omega_1B)
      #print("O2B=", omega_2B)
      
      y_omega = np.append([omega_0B], np.append(reshape(omega_1B, -1), reshape(omega_2B, -1)))
      y_ham = np.append([E0], np.append(reshape(f0, -1), reshape(Gamma0, -1)))  
      #print(y_ham[0:64])
      #print(y_omega[0:64])
        
      #ham = np.append([E0], np.append(reshape(f0, -1), reshape(Gamma0, -1)))
      ham = np.zeros_like(y_ham)
      
      for j in range(6):
        ham += (1./factorial(j)) * recursive_func(y_omega, y_ham, j)
#         print(ham[0])
#         print(ham[1:16])
#       print(ham[0])
#       print(np.linalg.norm(ham))
        
      E, f, Gamma = get_operator_from_y(ham, dim1B, dim2B) 
      
      eta1B, eta2B = eta_wegner(f, Gamma, user_data)
      
    
#       print(np.linalg.norm(eta2B))
#       print(np.linalg.norm(eta2B+eta2B.T))
        
      y_eta   = np.append([0.0], np.append(reshape(eta1B, -1), reshape(eta2B, -1)))  
      
      # initialization of omega and ham
      #domega = np.append([omega_0B], np.append(reshape(omega_1B, -1), reshape(omega_2B, -1)))
      
      h = 0.025
      domega = np.zeros_like(y_omega)
      #print("shape of y_omega=", y_omega.shape)
      #print("shape of y_eta=", y_eta.shape)
      #print("shape of y_ham=", y_ham.shape)
    
      for i in range(deriv_value):        
        domega += (bernoulli_arr[i]/factorial(i)) * recursive_func_alt(y_omega, y_eta, i)
        #print("domega=", domega[1:2*dim1B])
        #print("domega2B=", domega[dim1B*dim1B+2:dim1B*dim1B+2+2*dim2B])
        #print("y_eta2B=", y_eta[dim1B*dim1B+2:dim1B*dim1B+2+2*dim2B])
        
      tmp = y_omega
      y_omega = y_omega + h * domega
      
            
      return y_omega, ham[0]

In [23]:
deriv_value = 4
amount = 75
y_omega_arr = []

E0, f0, Gamma0 = normal_order(H1B, H2B, user_data)   
print(E0)


y_omega = np.zeros(1+dim1B*dim1B+dim2B*dim2B)
for j in range(amount):
    y_omega, E = magnus_stepper(y_omega, E0, f0, Gamma0, deriv_value)
    y_omega_arr.append(y_omega.copy())
    print(E)
    
#print(y_omega_arr)

1.5
1.5
1.4663581162799202
1.4522620217402014
1.4437663466473056
1.4378759969593835
1.433424183669969
1.4298995331791764
1.4270433228414086
1.4247021733686458
1.4227720889222866
1.4211759437956302
1.4198535486755441
1.418756655422548
1.417846045477582
1.4170895993510484
1.416460896478876
1.415938143723806
1.4155033304010414
1.4151415506608536
1.4148404546880342
1.4145898014272775
1.414381092466129
1.414207271385738
1.4140624762574991
1.413941835485959
1.4138412991350593
1.4137574993867004
1.413687634974836
1.4136293753901628
1.4135807814145782
1.4135402391616212
1.4135064052999056
1.4134781615446368
1.4134545768360718
1.4134348758975337
1.413418413090708
1.413404650671432
1.4133931407022995
1.4133835100049703
1.413375447639821
1.4133686944873598
1.4133630345777661
1.4133582878746367
1.4133543042685646
1.4133509585773425
1.4133481463837994
1.4133457805706733
1.4133437884355986
1.413342109288883
1.413340692453144
1.4133394955974286
1.4133384833497609
1.4133376261414794
1.4133368992445259